In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
if torch.cuda.is_available():
  model_id = "meta-llama/Llama-2-7b-chat-hf"
  model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
  model.cuda()
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.use_default_system_prompt = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [ ]:
# # Model from Hugging Face hub
# base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
indi_dataset = "Mukesh555/indian_lawyer_dataset"

# Fine-tuned model
# new_model = "llama-2-7b-chat-guanaco"

In [ ]:
print(indi_dataset)

In [ ]:
dataset = load_dataset(indi_dataset, split="train")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

In [ ]:
print(dataset['instruction'])

In [ ]:
print(dataset['output'])

In [ ]:
# from sklearn.model_selection import train_test_split
# from datasets import Dataset

In [ ]:
# # Assuming your dataset is named 'my_dataset'
# # Replace the next line with your actual dataset

# my_dataset = Dataset({
#     'features': {
#         'instruction': dataset['instruction'],  # Replace with your actual 'instruction' data
#         'output': dataset['output']        # Replace with your actual 'output' data
#     },
#     'num_rows': 1000
# })

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=quant_config,
#     device_map={"": 0}
# )
# model.config.use_cache = False
# model.config.pretraining_tp = 1

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="instruction",
    max_seq_length=50,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

Step,Training Loss
25,2.062000
50,0.849600
75,0.815600
100,0.485500
125,0.587600
150,0.389800
175,0.485500
200,0.320900
225,0.437900
250,0.252200


TrainOutput(global_step=250, training_loss=0.6686737003326416, metrics={'train_runtime': 89.5686, 'train_samples_per_second': 11.165, 'train_steps_per_second': 2.791, 'total_flos': 1229628682567680.0, 'train_loss': 0.6686737003326416, 'epoch': 1.0})

In [ ]:
pip install --upgrade huggingface_hub

In [ ]:
login()

In [ ]:
model.push_to_hub('Chandana03/Llama-2-7b-chat-hf.legal-FineTune')

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.61G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chandana03/Llama-2-7b-chat-hf.legal-FineTune/commit/2b1bc338038a516d97f749ad6be53141f4b6f429', commit_message='Upload LlamaForCausalLM', commit_description='', oid='2b1bc338038a516d97f749ad6be53141f4b6f429', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub('Chandana03/Llama-2-7b-chat-hf.legal-FineTune')

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chandana03/Llama-2-7b-chat-hf.legal-FineTune/commit/ae760ed90da3b9b39be592b1c5f02c782eb38e7f', commit_message='Upload tokenizer', commit_description='', oid='ae760ed90da3b9b39be592b1c5f02c782eb38e7f', pr_url=None, pr_revision=None, pr_num=None)

#**Loading finetuned model**

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizerF = AutoTokenizer.from_pretrained("Chandana03/Llama-2-7b-chat-hf.legal-FineTune")
modelF = AutoModelForCausalLM.from_pretrained("Chandana03/Llama-2-7b-chat-hf.legal-FineTune")

tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/37.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.61G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]